In [ ]:
pip install anthropic

### Setup your own API key
* Login & setup your API key from here (https://console.anthropic.com/settings/keys)
* Paste your API key in the following inverted commas

In [ ]:
import os
# Set the environment variable within the notebook
os.environ["ANTHROPIC_API_KEY"] = "API_KEY"

***Following code is just a sample, your task is to encapsulate this code in a function with arguments of your choice so that function can be reused***

#### Following link is for reference: https://docs.anthropic.com/en/docs/initial-setup#next-steps

In [ ]:
import anthropic

client = anthropic.Anthropic()

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=0,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)
print(message.content)


In [ ]:
from groq import Groq
client = Groq(
    api_key="API_KEY"
)

In [ ]:
import os
import csv
import json
import time
from groq import Groq


reviews=[]
with open('reviews.csv', mode='r', encoding='utf-8') as file:
    reader=csv.DictReader(file)
    for row in reader:
        reviews.append(row)


def categorize_review(review_text):
    prompt_food=f"""
    Please extract only the comments that specifically describe the quality of the food in the following review. 
    Do **not** include any comments about the service, staff, or anything unrelated to the food. 
    Be specific about how the food tasted, the presentation, or the overall dining experience related to the food itself.
    Here is the review: "{review_text}"
    """

    prompt_service=f"""
    Please extract only the comments that specifically describe the staff or service in the following review. 
    Do **not** include any comments about the food, taste, or anything unrelated to the service or staff. 
    Be specific about the behavior, attentiveness, professionalism, and attitude of the staff during the dining experience.
    Here is the review: "{review_text}"
    """
    
    food_response=client.chat.completions.create(
        messages=[{"role": "user", "content": prompt_food}],
        model="llama3-8b-8192"
    )
    
    service_response=client.chat.completions.create(
        messages=[{"role": "user", "content": prompt_service}],
        model="llama3-8b-8192"
    )
    
    food_comments=food_response.choices[0].message.content.strip()
    service_comments=service_response.choices[0].message.content.strip()

    print("Food Comments: ", food_comments)
    print("Service Comments: ", service_comments)
    
    return {
        "food_comments": food_comments,
        "service_comments": service_comments
    }


with open('processed_reviews.json', 'w', encoding='utf-8') as json_file:
    
    json_file.write('[')


processed_reviews=[]


batch_size=30
total_reviews=len(reviews)

for i in range(0, total_reviews, batch_size):
    batch=reviews[i:i + batch_size]
    
    for review in batch:
        review_text=review['review_text']
        
        categorized_data=categorize_review(review_text)
        
        categorized_data["food_comments"]=categorized_data["food_comments"].replace("personal information", "")
        categorized_data["service_comments"]=categorized_data["service_comments"].replace("personal information", "")
        
        processed_review={
            "restaurant_name": review['restaurant_name'],
            "review_rating": review['review_rating'],
            "food_comments": categorized_data["food_comments"],
            "service_comments": categorized_data["service_comments"]
        }

        with open('processed_reviews.json', 'a', encoding='utf-8') as json_file:
            if i > 0 or len(processed_reviews) > 0:
                json_file.write(',\n')
            json.dump(processed_review, json_file, ensure_ascii=False, indent=4)

    print(f"Processed batch {i // batch_size + 1} of {total_reviews // batch_size + 1}. Waiting for 60 seconds...")
    #time.sleep(60)

with open('processed_reviews.json', 'a', encoding='utf-8') as json_file:
    if i == 0 and len(processed_reviews) == 0:
        json_file.write('[')  
    elif i > 0 or len(processed_reviews) > 0:
        json_file.write(',\n') 

    json.dump(processed_review, json_file, ensure_ascii=False, indent=4)

with open('processed_reviews.json', 'a', encoding='utf-8') as json_file:
    json_file.write('\n]')

